In [ ]:

# Cell 1: Install Dependencies
!pip install torch transformers langchain faiss-cpu sentence-transformers gradio huggingface_hub

# Cell 2: Login to Hugging Face
from huggingface_hub import login

# Replace with your actual Hugging Face token
hf_token = "hf token"  # Paste your token here
login(token=hf_token)
print("Logged in to Hugging Face successfully!")

# Cell 3: Download Mistral Model
import os
import torch  # Added torch import here
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "mistralai/Mistral-7B-v0.1"
model_path = "/content/models/mistral"  # Colab's temporary storage

# Create directory
os.makedirs(model_path, exist_ok=True)

print("Downloading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained(model_path)

print("Downloading model... (this may take a while)")
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.save_pretrained(model_path)

print(f"Model saved to {model_path}")

# Cell 4: Chatbot Setup and Launch
# All imports consolidated here
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
import gradio as gr

# Load the tokenizer and model from the local directory
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.float16)

text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    device=0 if torch.cuda.is_available() else -1
)

llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

# Setup FAISS and Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

docs = [
    "It's okay to feel down sometimes. You're not alone.",
    "Try taking deep breaths or a short walk to help clear your mind.",
    "If you're feeling overwhelmed, consider talking to someone you trust.",
    "Mental health matters. Be kind to yourself."
]

documents = [Document(page_content=d) for d in docs]
splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = splitter.split_documents(documents)

vectordb = FAISS.from_documents(split_docs, embedding_model)

# Retrieval Chain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 2}),
    memory=memory,
    return_source_documents=False
)

# Agent Tool Setup
tools = [
    Tool(
        name="MentalHealthSupport",
        func=qa_chain.run,
        description="Helpful mental health assistant"
    )
]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    verbose=True
)

# Gradio Interface
def chat_interface(message, history):
    try:
        return agent.run(message)
    except Exception as e:
        return f"Error: {e}"

chatbot_ui = gr.ChatInterface(fn=chat_interface,
                              title="🧠 Mental Health Chatbot",
                              description="Private, local, empathetic chatbot powered by Mistral")

# Launch
chatbot_ui.launch()

Logged in to Hugging Face successfully!


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
!pip install torch